In [7]:
print('ping')

ping


# Process All Spectra

(Can get around memory limit in Jupyter Notebook)

In [3]:
'''
Code for reading in 1-d DEIMOS spectra reduced by the spec2d code:
http://deep.ps.uci.edu/spec2d/. Can process spectra from a single target
or from a list of targets sequentially.

This code:
1) Reads in 1-d DEIMOS spectra .fits files and concatenates spectra from the
red and blue CCDs.
2) Interpolates spectra onto the standard wavelength template for DEIMOS.
3) Outputs all object spectra in a .npz file.
'''

from __future__ import absolute_import, division, print_function
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import multiprocessing
import utils

# # # Settings # # #


# DEIMOS Directory
DEIMOSDir = '/global/scratch/nathan_sandford/DEIMOS/U112_reduced/'
#DEIMOSDir = '/global/scratch/nathan_sandford/DEIMOS/Sgr_dSph/'

# D-Payne Directory
D_PayneDir = utils.D_PayneDir

# Output directory for processed spectra
OutputDir = DEIMOSDir+'obs_spectra/'

'''
List of targets to process.
Note that 'm92' is missing and 'pegdig' includes data from two nights.
''' 
#  already done
# '211SgS', '212SgS', '219SgS' already done
targets = ['m2', 'm13', 'm15','m71', 'ngc2419', 'ngc7006', 'pegdig', 'pegdig24', 'pegdig26', 'wlm']

'''
Which method of fitting the DEIMOS spectrum do you want to use:
'Horne' or 'Bxspf'
'''
method = 'Horne'

'''
Do you want to plot cross correlation function?
'''
plot_xcorr_flag = False

# Restore Wavelength Template
print('Restoring Wavelength Template')
wavelength_template = utils.load_wavelength_array()

# Calculate matrix of distances between wavelengths
print('Calculating matrix of distances between wavelengths...')
wavelength_diff_matrix \
    = wavelength_template[:, np.newaxis] - wavelength_template

# Restore DEIMOS continuum pixels
print('Restoring DEIMOS continuum pixels')
cont_reg = utils.load_deimos_cont_pixels()

# Restore telluric mask from Kirby+ 2008
print('Restoring telluric mask from Kirby+ 2008...')
kirby_2008_telluric = utils.get_spectral_mask_dict(name='kirby_2008_telluric')
mask = utils.generate_mask_from_dict(**kirby_2008_telluric)


def get_deimos_spectra(Obj, method, InputDir=None):
    '''
    Read in and concatenate red and blue spectra
    '''
    ObjHDUL = fits.open(InputDir + Obj)
    
    

    try:
        waveB = ObjHDUL[method+'-B'].data['LAMBDA'][0]
        blue = [np.min(waveB), np.max(waveB)]
        waveR = ObjHDUL[method+'-R'].data['LAMBDA'][0]
        red = [np.min(waveR), np.max(waveR)]
        wave = np.concatenate((waveB, waveR))
    
        specB = ObjHDUL[method+'-B'].data['SPEC'][0]
        specR = ObjHDUL[method+'-R'].data['SPEC'][0]
        spec = np.concatenate((specB, specR))
    
        ivarB = ObjHDUL[method+'-B'].data['IVAR'][0]
        ivarR = ObjHDUL[method+'-R'].data['IVAR'][0]
        ivar = np.concatenate((ivarB, ivarR))
        ivar[ivar == 0] = 1e-16  # Avoid np.inf in spec_err
    except KeyError:
        print('%s Missing %s spectra' % (Obj, method))
        wave, spec, ivar = np.zeros((3,len(wavelength_template)))
        blue, red = [0, 0], [0, 0]
        
    SlitNumber = np.int(Obj[14:17])
    RA_Dec = RA_Dec_msk[SlitNumber]
        
    return (wave, spec, ivar, RA_Dec, blue, red)


def interpolate_deimos_spectra(wave, spec, spec_err):
    '''
    Interpolates a DEIMOS spectrum onto the default wavelength grid
    '''
    if len(wave) != 16250:
        spec = np.interp(wavelength_template, wave, spec)
        spec_err = np.interp(wavelength_template, wave, spec_err)
        wave = np.copy(wavelength_template)
    return(wave, spec, spec_err)


def fast_RV(Obj, spec, spec_err, plot=False):
    '''
    Quick radial velocity determination by cross-correlating observed spectrum
    with a template spectrum in the region around the Ca II triplet.
    '''

    # Consider only region around Ca II triplet
    temp_spec = spec[(wavelength_template > 8450) &
                     (wavelength_template < 8700)]
    temp_spec_err = spec_err[(wavelength_template > 8450) &
                             (wavelength_template < 8700)]

    # Cross-Correlate
    num = np.sum(template_grid * temp_spec / temp_spec_err, axis=1)
    den = np.sum(template_grid * template_grid / temp_spec_err, axis=1)
    xcorr = num / den

    # Naively take maximum of CCF
    RV = RV_grid[np.argmax(xcorr)]

    # Plot output
    if plot_xcorr_flag:
        plt.plot(RV_grid, xcorr)
        plt.xlabel('RV (km/s)')
        plt.ylabel('Cross-Corellation')
        plt.title('%s' % Obj)
        plt.show()

    return(RV)


def process_deimos_spectra(i):
    '''
    Processes all spectra in InputList
    '''
    Obj = ObjList[i]
    ObjNumber = Obj[14:-5]
    print('Processing spectra for object: %s' % Obj)

    print('Restoring spectra #%s' % ObjNumber)
    wave_temp, spec_temp, ivar_temp, RA_Dec, blue, red = \
        get_deimos_spectra(Obj=Obj, method=method, InputDir=InputDir)

    print('Interpolating spectra #%s' % ObjNumber)
    wavelength, spec, spec_err = \
        interpolate_deimos_spectra(wave=wave_temp,
                                   spec=spec_temp,
                                   spec_err=(ivar_temp**-1))

    print('Applying telluric mask for spectra %s' % ObjNumber)
    spec_err[mask] = 1e16

    print('Finding radial velocity from CCF for spectra #%s' % ObjNumber)
    RV = fast_RV(Obj, spec, spec_err, plot=plot_xcorr_flag)
    print('%s has Radial Velocity = %.0f' % (ObjNumber, RV))
    print('Shifting spectra %s to rest frame' % ObjNumber)
    spec = utils.doppler_shift(wavelength=wavelength, flux=spec, RV=-RV)

    print('Normalizing spectra %s' % ObjNumber)
    try:
        cont_spec_blue = \
            utils.get_deimos_continuum_poly(spec, spec_err=spec_err,
                                            wavelength=wavelength,
                                            cont_pixels=cont_reg, bounds=blue,
                                            deg=6, verbose=False)
        cont_spec_red = \
            utils.get_deimos_continuum_poly(spec, spec_err=spec_err,
                                            wavelength=wavelength,
                                            cont_pixels=cont_reg, bounds=red,
                                            deg=6, verbose=False)
    except TypeError:
        print('Normalization failed')
        cont_spec_blue = np.ones(len(wavelength_template))
        cont_spec_red = np.ones(len(wavelength_template))
    norm_spec_blue = spec[wavelength_template < red[0]] / \
        cont_spec_blue[wavelength_template < red[0]]
    norm_spec_red = spec[wavelength_template > red[0]] / \
        cont_spec_blue[wavelength_template > red[0]]
    norm_spec = np.concatenate((norm_spec_blue, norm_spec_red))

    # Handle regions where continuum is zero
    spec_err[np.isnan(norm_spec)] = 1e16
    norm_spec[np.isnan(norm_spec)] = 0
    
    return(ObjNumber, wavelength, spec, norm_spec, spec_err, RV, RA_Dec)


# Restore spectral template of typical RGB star
print('Restoring spectral template...')
temp = np.load(D_PayneDir + '/other_data/typical_RGB_spectrum.npz')
wave, template_spec, spec_err = interpolate_deimos_spectra(temp['wavelength'], temp['spectra'], np.zeros(len(temp['spectra'])))
template_spec = template_spec[(wavelength_template > 8450) &
                             (wavelength_template < 8700)]
temp.close()

# Prepping cross-correlation function
print('Prepping cross-correlation calculation...')
wavelength_CaIItriplet = wavelength_template[(wavelength_template > 8450) &
                                             (wavelength_template < 8700)]
RV_grid = np.linspace(-500, 250, 750)
template_grid = np.empty((len(RV_grid), len(wavelength_CaIItriplet)))
for i, RV in enumerate(RV_grid):
    template_grid[i] = utils.doppler_shift(wavelength=wavelength_CaIItriplet,
                                           flux=template_spec, RV=RV)

    
for target in targets:
    print('Beginning reduction for %s' % target)
    
    # RA + Dec of slits in this targeting
    bintabs = fits.open(DEIMOSDir + '/' + target + '/' + target + '.bintabs.fits')
    RA = bintabs['ObjectCat'].data['RA_OBJ']
    Dec = bintabs['ObjectCat'].data['DEC_OBJ']
    RA_Dec_msk = SkyCoord(RA, Dec, unit=(u.deg, u.deg))
    
    # Input List of DEIMOS spectra to be processed for fitting
    InputDir = DEIMOSDir + target + '/'
    
    # InputList = InputDir + 'spec1d.'+target+'msk.txt'
    InputList = InputDir + 'spec1d.'+target+'.txt'
    
    # File containing processed spectra
    OutputFile = target+'_'+method+'.npz'
    
    # Extract object names from input list
    print('Restoring list of %s objects' % target)
    temp = open(InputList, 'r')
    ObjList = temp.read().split('\n')[:-2]
    temp.close()
    
    # Process the spectra
    print('Beginning processing of all spectra for %s' % target)
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    temp = pool.map(process_deimos_spectra, range(len(ObjList)))
    temp = list(zip(*temp))
    ObjNumber, wavelength, spec, norm_spec, spec_err, RV, RA_Dec = temp
    print('Completed processing of all spectra')
    
    # Save processed spectra
    print('Saving all processed spectra for %s to %s\n\n\n\n\n' % (target,OutputFile))
    np.savez(OutputDir + OutputFile, obj=ObjNumber, wavelength=wavelength,
             spec=spec, norm_spec=norm_spec, spec_err=spec_err, RV=RV, RA_Dec=RA_Dec)
    

Restoring Wavelength Template
Calculating matrix of distances between wavelengths...
Restoring DEIMOS continuum pixels
Restoring telluric mask from Kirby+ 2008...
Restoring spectral template...
Prepping cross-correlation calculation...
Beginning reduction for 211SgS
Restoring list of 211SgS objects
Beginning processing of all spectra for 211SgS
Processing spectra for object: spec1d.211SgS.000.163Sgr.fits
Processing spectra for object: spec1d.211SgS.002.163Sgr.fits
Processing spectra for object: spec1d.211SgS.004.163Sgr.fits
Processing spectra for object: spec1d.211SgS.006.163Sgr.fits
Processing spectra for object: spec1d.211SgS.008.163Sgr.fits
Processing spectra for object: spec1d.211SgS.010.163Sgr.fits
Restoring spectra #002.163Sgr
Restoring spectra #000.163Sgr
Processing spectra for object: spec1d.211SgS.012.163Sgr.fits
Processing spectra for object: spec1d.211SgS.013.163Sgr.fits
Processing spectra for object: spec1d.211SgS.015.163Sgr.fits
Restoring spectra #004.163Sgr
Processing spe

Finding radial velocity from CCF for spectra #017.163Sgr
Processing spectra for object: spec1d.211SgS.044.163Sgr.fits
017.163Sgr has Radial Velocity = 101
Processing spectra for object: spec1d.211SgS.042.163Sgr.fits
Processing spectra for object: spec1d.211SgS.060.163Sgr.fits
Restoring spectra #044.163Sgr
Shifting spectra 017.163Sgr to rest frame
Processing spectra for object: spec1d.211SgS.063.serendip1.fits
Restoring spectra #042.163Sgr
Processing spectra for object: spec1d.211SgS.073.163Sgr.fits
Restoring spectra #060.163Sgr
Processing spectra for object: spec1d.211SgS.019.163Sgr.fits
Processing spectra for object: spec1d.211SgS.025.163Sgr.fits
Restoring spectra #063.serendip1
Normalizing spectra 017.163Sgr
Processing spectra for object: spec1d.211SgS.014.163Sgr.fits
Processing spectra for object: spec1d.211SgS.062.163Sgr.fits
Restoring spectra #073.163Sgr
Restoring spectra #019.163Sgr
Processing spectra for object: spec1d.211SgS.052.163Sgr.fits
Processing spectra for object: spec1d

Shifting spectra 014.163Sgr to rest frame
Restoring spectra #035.163Sgr
Interpolating spectra #017.serendip1
Processing spectra for object: spec1d.211SgS.040.163Sgr.fits
Processing spectra for object: spec1d.211SgS.036.serendip1.fits
Normalizing spectra 014.163Sgr
Processing spectra for object: spec1d.211SgS.085.163Sgr.fits
Restoring spectra #040.163Sgr
Applying telluric mask for spectra 017.serendip1
Restoring spectra #085.163Sgr
Interpolating spectra #001.163Sgr
Finding radial velocity from CCF for spectra #017.serendip1
Applying telluric mask for spectra 001.163Sgr
017.serendip1 has Radial Velocity = 77
Processing spectra for object: spec1d.211SgS.089.163Sgr.fits
Processing spectra for object: spec1d.211SgS.087.163Sgr.fits
Processing spectra for object: spec1d.211SgS.056.163Sgr.fits
Finding radial velocity from CCF for spectra #001.163Sgr
Shifting spectra 017.serendip1 to rest frame
Processing spectra for object: spec1d.211SgS.030.163Sgr.fits
Restoring spectra #089.163Sgr
Restoring 

Finding radial velocity from CCF for spectra #089.163Sgr
Applying telluric mask for spectra 036.serendip1
Finding radial velocity from CCF for spectra #040.163Sgr
087.163Sgr has Radial Velocity = 82
021.163Sgr has Radial Velocity = 81
091.163Sgr has Radial Velocity = 80
Normalizing spectra 083.163Sgr
Interpolating spectra #093.163Sgr
Applying telluric mask for spectra 033.163Sgr
089.163Sgr has Radial Velocity = 79
040.163Sgr has Radial Velocity = 78
Interpolating spectra #071.163Sgr
Finding radial velocity from CCF for spectra #036.serendip1
Shifting spectra 087.163Sgr to rest frame
Applying telluric mask for spectra 093.163Sgr
Processing spectra for object: spec1d.211SgS.069.163Sgr.fits
Shifting spectra 040.163Sgr to rest frame
Finding radial velocity from CCF for spectra #033.163Sgr
Normalizing spectra 087.163Sgr
Finding radial velocity from CCF for spectra #093.163Sgr
Shifting spectra 091.163Sgr to rest frame
Processing spectra for object: spec1d.211SgS.110.163Sgr.fits
085.163Sgr ha

/global/home/users/nathan_sandford/D-Payne/utils.py:151: RuntimeWarning: invalid value encountered in sqrt
  deg=deg, w=np.sqrt(spec_err[fit_reg])**-1)


...

Interpolating spectra #052.219Sgr
Restoring spectra #088.219Sgr
Interpolating spectra #072.219Sgr
Interpolating spectra #099.219Sgr
Applying telluric mask for spectra 072.219Sgr
Applying telluric mask for spectra 052.219Sgr
Shifting spectra 024.serendip1 to rest frame
Shifting spectra 097.219Sgr to rest frame
Finding radial velocity from CCF for spectra #052.219Sgr
Processing spectra for object: spec1d.219SgS.130.219Sgr.fits
052.219Sgr has Radial Velocity = -437
Restoring spectra #130.219Sgr
Shifting spectra 052.219Sgr to rest frame
Shifting spectra 095.219Sgr to rest frame
Normalizing spectra 054.219Sgr
Normalizing spectra 052.219Sgr
Restoring spectra #128.219Sgr
Normalizing spectra 024.serendip1
Normalizing spectra 097.219Sgr
Processing spectra for object: spec1d.219SgS.084.219Sgr.fits
Normalizing spectra 095.219Sgr
Restoring spectra #084.219Sgr
Interpolating spectra #092.219Sgr
Processing spectra for object: spec1d.219SgS.132.219Sgr.fits
Applying telluric mask for spectra 092.219Sgr

132.219Sgr has Radial Velocity = 65
Processing spectra for object: spec1d.219SgS.100.219Sgr.fits
Applying telluric mask for spectra 130.219Sgr
Restoring spectra #156.219Sgr
Processing spectra for object: spec1d.219SgS.125.219Sgr.fits
Restoring spectra #109.219Sgr
Normalizing spectra 128.219Sgr
Finding radial velocity from CCF for spectra #130.219Sgr
Shifting spectra 098.219Sgr to rest frame
130.219Sgr has Radial Velocity = 72
Restoring spectra #100.219Sgr
078.219Sgr has Radial Velocity = 169
Processing spectra for object: spec1d.219SgS.122.219Sgr.fits
Shifting spectra 130.219Sgr to rest frame
Shifting spectra 078.219Sgr to rest frame
Restoring spectra #123.serendip1
Normalizing spectra 098.219Sgr
Restoring spectra #122.219Sgr
Normalizing spectra 130.219Sgr
Shifting spectra 132.219Sgr to rest frame
Normalizing spectra 078.219Sgr
Processing spectra for object: spec1d.219SgS.107.219Sgr.fits
Processing spectra for object: spec1d.219SgS.112.219Sgr.fits
Interpolating spectra #082.219Sgr
Rest

Interpolating spectra #156.219Sgr
Normalizing spectra 103.serendip1
109.219Sgr has Radial Velocity = 74
115.219Sgr has Radial Velocity = 73
Processing spectra for object: spec1d.219SgS.169.219Sgr.fits
Restoring spectra #173.219Sgr
Shifting spectra 084.219Sgr to rest frame
Shifting spectra 115.219Sgr to rest frame
Normalizing spectra 084.219Sgr
Shifting spectra 109.219Sgr to rest frame
Applying telluric mask for spectra 156.219Sgr
Applying telluric mask for spectra 123.serendip1
Processing spectra for object: spec1d.219SgS.143.219Sgr.fits
Processing spectra for object: spec1d.219SgS.154.219Sgr.fits
Normalizing spectra 109.219Sgr
Normalizing spectra 115.219Sgr
Processing spectra for object: spec1d.219SgS.175.219Sgr.fits
Processing spectra for object: spec1d.219SgS.176.serendip1.fits
Finding radial velocity from CCF for spectra #156.219Sgr
Processing spectra for object: spec1d.219SgS.171.219Sgr.fits
Restoring spectra #143.219Sgr
Processing spectra for object: spec1d.219SgS.178.219Sgr.fits

Normalizing spectra 163.219Sgr
Shifting spectra 129.219Sgr to rest frame
Shifting spectra 147.219Sgr to rest frame
Shifting spectra 165.219Sgr to rest frame
Shifting spectra 171.219Sgr to rest frame
Normalizing spectra 178.219Sgr
Interpolating spectra #143.219Sgr
Normalizing spectra 176.serendip1
Normalizing spectra 147.219Sgr
Applying telluric mask for spectra 173.219Sgr
Normalizing spectra 171.219Sgr
Normalizing spectra 165.219Sgr
Normalizing spectra 129.219Sgr
Interpolating spectra #182.219Sgr
Interpolating spectra #160.219Sgr
Shifting spectra 136.219Sgr to rest frame
Normalizing spectra 136.219Sgr
Finding radial velocity from CCF for spectra #173.219Sgr
Applying telluric mask for spectra 182.219Sgr
Finding radial velocity from CCF for spectra #182.219Sgr
Interpolating spectra #184.219Sgr
182.219Sgr has Radial Velocity = 111
Applying telluric mask for spectra 184.219Sgr
161.serendip1 has Radial Velocity = -500
Finding radial velocity from CCF for spectra #184.219Sgr
Finding radial v

Finding radial velocity from CCF for spectra #181.219Sgr
Interpolating spectra #174.219Sgr
177.219Sgr has Radial Velocity = -209
181.219Sgr has Radial Velocity = 72
Applying telluric mask for spectra 174.219Sgr
Shifting spectra 177.219Sgr to rest frame
Finding radial velocity from CCF for spectra #174.219Sgr
Normalizing spectra 177.219Sgr
174.219Sgr has Radial Velocity = -348
Shifting spectra 183.219Sgr to rest frame
Shifting spectra 181.219Sgr to rest frame
Normalizing spectra 181.219Sgr
Normalizing spectra 183.219Sgr
Shifting spectra 174.219Sgr to rest frame
Normalizing spectra 174.219Sgr
Interpolating spectra #164.219Sgr
Applying telluric mask for spectra 164.219Sgr
Finding radial velocity from CCF for spectra #164.219Sgr
164.219Sgr has Radial Velocity = 71
Shifting spectra 164.219Sgr to rest frame
Normalizing spectra 164.219Sgr
Interpolating spectra #166.219Sgr
Interpolating spectra #187.219Sgr
Applying telluric mask for spectra 166.219Sgr
Applying telluric mask for spectra 187.219

In [10]:
# DEIMOS Directory
DEIMOSDir = '/global/scratch/nathan_sandford/DEIMOS/U112_reduced/'
#DEIMOSDir = '/global/scratch/nathan_sandford/DEIMOS/Sgr_dSph/'

# D-Payne Directory
D_PayneDir = utils.D_PayneDir

# Output directory for processed spectra
OutputDir = DEIMOSDir+'obs_spectra/'

targets = ['m2', 'm13', 'm15','m71', 'ngc2419', 'ngc7006', 'pegdig', 'pegdig24', 'pegdig26', 'wlm']

In [12]:
for target in targets:
    print('Beginning reduction for %s' % target)
    
    # RA + Dec of slits in this targeting
    bintabs = fits.open(DEIMOSDir + target + '/' + target + '.bintabs.fits')
    RA = bintabs['ObjectCat'].data['RA_OBJ']
    Dec = bintabs['ObjectCat'].data['DEC_OBJ']
    RA_Dec_msk = SkyCoord(RA, Dec, unit=(u.deg, u.deg))
    
    # Input List of DEIMOS spectra to be processed for fitting
    InputDir = DEIMOSDir + target + '/'
    
    # InputList = InputDir + 'spec1d.'+target+'msk.txt'
    InputList = InputDir + 'spec1d.'+target+'.txt'
    
    # File containing processed spectra
    OutputFile = target+'_'+method+'.npz'
    
    # Extract object names from input list
    print('Restoring list of %s objects' % target)
    temp = open(InputList, 'r')
    ObjList = temp.read().split('\n')[:-2]
    temp.close()

Beginning reduction for m2


FileNotFoundError: [Errno 2] No such file or directory: '/global/scratch/nathan_sandford/DEIMOS/U112_reduced/m2/m2.bintabs.fits'